In [1]:
from pyspark.sql import SparkSession
from datetime import datetime, timezone
import re
from pyspark.sql.functions import col, udf, size
from pyspark.sql.types import StringType, ArrayType
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.242:7077") \
        .appName("reddit_analysis_t11")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.eventLog.enabled", True)\
        .config("spark.eventLog.dir", "hdfs://192.168.2.242:9000/user/shared/spark-logs")\
        .config("spark.cores.max", 8)\
        .getOrCreate()

# Old API (RDD) 
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

In [2]:
#cores = 10
df = spark_session.read.json("hdfs://192.168.2.242:9000/user/reddit_data/big/strong_scaling/*")
#df.coalesce(cores*2)

#print("Input Sample:\n\n", df.take(2))
#print("\nNumber of partitions:", df.rdd.getNumPartitions())
#print("row entries count:", df.count())

#print("\nReddit post schema:\n")
#df.printSchema()

# test converting timestamp
first_timestamp = datetime.fromtimestamp(df.first()["created_utc"], tz=timezone.utc)
last_timestamp = datetime.fromtimestamp(df.tail(1)[0]["created_utc"], tz=timezone.utc)

#print("First Timestamp of this batch: ", first_timestamp)
#print("Last Timestamp of this batch", last_timestamp)


start_ts = int(datetime(2000, 1, 1, 0, 0).timestamp())
end_ts = int(datetime(2030, 1, 31, 0, 0).timestamp())
date_range = range(start_ts, end_ts)
#print("start_ts:", start_ts)
#print("end_ts:", end_ts)
df = df[['created_utc', 'body']]

In [3]:
# drop everything else
#in_range = df[['created_utc', 'body']]
#print(type(df.first()["created_utc"]))
df = df.filter((end_ts >= df.created_utc) & (df.created_utc >= start_ts))
df = df.filter(df.body != '[deleted]')
df = df.filter(df.body != '[removed]')
df.cache()
#in_range.cache()
#print(type(df))
#print("count after filter on date:", df.count())
#print(df.created_utc)

DataFrame[created_utc: bigint, body: string]

In [4]:
def normalize(string):
    return  re.sub(r'[^A-Za-z ]', '', string.lower())

udf_normalize = udf(normalize, StringType())

df = df.withColumn('body', udf_normalize(col('body')))
#print(df.select("body").take(10))

#posts = in_range.select("body").take(10)
#single_post = posts[0]
# normalize, remove special signs and lower case strings
#list_posts = [re.sub(r'[^A-Za-z ]', '', post["body"].lower()) for post in posts]
#print(list_posts[0:10])


In [ ]:
pres_cand = ["donald trump", "hillary clinton"]
#print(df.take(1))
""" 
data_rdd = df.rdd\
             .map(lambda line: tuple(set([cand for cand in pres_cand if cand in line[1]])))\
             .filter(lambda match: match != ())\
             .map(lambda word: (word, 1)).reduceByKey(add)
"""

def return_candidate(string):
    present = [cand for cand in pres_cand if cand in string]
    if present == []:
        present = ''
    return str(present)

udf_rc = udf(return_candidate, StringType())
# df alternative
df = df.withColumn('body', udf_rc(col('body')))
df = df.filter(df.body != '')
df = df.groupBy('body').count()
df.repartition(1).write.csv('save.csv')


In [ ]:
#spark_context.stop()